In [0]:
import numpy as np
import cv2
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from sklearn import metrics

from skimage import io, color

import time
import os
import pickle

import matplotlib.pyplot as plt
import scikitplot as skplt

In [0]:
# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


## ResNet

In [0]:
# define the model
resnet152 = models.resnet152(pretrained=True)
resnet152 = models.resnet152(num_classes=2)

finalconv_name = 'layer4'
model_name = 'resnet152'

# training process
model = resnet152

LOAD_MODEL = True
MODEL_PATH = 'best_models/resnet152_best_model_178.pth'

if LOAD_MODEL:
    model.load_state_dict(torch.load(MODEL_PATH))

model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [0]:
def returnCAM(feature_conv, weight_softmax, class_idx):
        # generate the class activation maps upsample to 256x256
        size_upsample = (256, 256)
        bz, nc, h, w = feature_conv.shape
        #output_cam = []
        #for idx in class_idx:
        cam = weight_softmax[class_idx].dot(feature_conv.reshape((nc, h*w)))
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        #output_cam.append(cv2.resize(cam_img, size_upsample))
        #print(len(output_cam))
        
        #return output_cam
        return cv2.resize(cam_img, size_upsample)
    
def hook_feature(module, input, output):
        features_blobs.append(output.data.cpu().numpy())

In [ ]:
val_csv = pd.read_csv("chest_xray_origin/all/val.csv")

In [0]:
for idx, val_class in enumerate(val_csv["class"]):
    print(idx, val_class)

0 normal
1 pneumonia
2 pneumonia
3 pneumonia
4 pneumonia
5 normal
6 pneumonia
7 pneumonia
8 normal
9 pneumonia
10 pneumonia
11 pneumonia
12 normal
13 pneumonia
14 pneumonia
15 pneumonia
16 normal
17 pneumonia
18 normal
19 normal
20 normal
21 pneumonia
22 pneumonia
23 pneumonia
24 normal
25 normal
26 pneumonia
27 pneumonia
28 normal
29 pneumonia
30 pneumonia
31 pneumonia
32 pneumonia
33 normal
34 pneumonia
35 pneumonia
36 normal
37 pneumonia
38 normal
39 normal
40 normal
41 normal
42 pneumonia
43 normal
44 pneumonia
45 normal
46 pneumonia
47 pneumonia
48 pneumonia
49 pneumonia
50 pneumonia
51 pneumonia
52 pneumonia
53 pneumonia
54 pneumonia
55 normal
56 pneumonia
57 pneumonia
58 pneumonia
59 pneumonia
60 pneumonia
61 pneumonia
62 pneumonia
63 normal
64 pneumonia
65 normal
66 pneumonia
67 pneumonia
68 pneumonia
69 pneumonia
70 normal
71 normal
72 pneumonia
73 pneumonia
74 normal
75 pneumonia
76 pneumonia
77 pneumonia
78 pneumonia
79 normal
80 pneumonia
81 normal
82 normal
83 normal
84 pn

In [0]:
# the index to be visualized
index_to_viz = 70

In [0]:
val_csv.loc[index_to_viz]

image_index    NORMAL2-IM-0276-0001.jpeg
class                             normal
label                                  0
Name: 70, dtype: object

In [0]:
# get the image filename for the image we want to visualize
#val_df_path = 'chest_xray_origin/val.csv'
image_root_dir = 'chest_xray_origin/all/'
image_name = val_csv.loc[index_to_viz]["image_index"]
image_filename = image_root_dir + image_name

image = io.imread(image_filename, as_gray=True)

validation_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize([364,364]),
        transforms.CenterCrop(320),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

# transform the image
# need to transpose: input size for ToPILImage() is H*W*C not C*H*W
image = np.repeat(image[None,...], 3, axis=0).transpose(1, 2, 0)
image = validation_transform(np.uint8(image))
# get the batch_size
image = image.unsqueeze(0)
image = image.to(device)

In [0]:
model.eval()

with torch.no_grad():

    # hook the feature extractor
    features_blobs = []
    
    model._modules.get(finalconv_name).register_forward_hook(hook_feature)

    # get the softmax weight
    params = list(model.parameters())
    # params[-2] since params[-1] is bias
    weight_softmax = np.squeeze(params[-2].data.cpu().numpy()) # linear weight we want to multiply with features

    # get the output so that we can get the activation maps
    output = model(image)
    #print("output", output)
    
    softmax = nn.Softmax(dim=1)

    output = softmax(output)
    #print(output)

    _, preds = torch.max(output, dim = 1)
    
    pred_labels = {0: "normal", 1: "pneumonia"}
    
    print("Predict label: {} (p = {:.4})".format(pred_labels[preds.item()], output.squeeze()[preds.item()]))
    print("True label: {}".format(val_csv.loc[index_to_viz]["class"]))

Predict label: normal (p = 0.9981)
True label: normal


In [0]:
# draw CAM
with torch.no_grad():
    
    CAMs = returnCAM(features_blobs[0], weight_softmax, preds.item())
    # render the CAM and output
    #print('output CAM' + str(j) + '.jpg for the top1 prediction: %s'%labels[idx_softmax[0]])
    #image_path = "chest_xray_origin/all/NORMAL2-IM-0059-0001.jpeg"
    img = cv2.imread(image_filename)
    height, width, _ = img.shape
    
    heatmap = cv2.applyColorMap(cv2.resize(CAMs,(width, height)), cv2.COLORMAP_JET)
    result = heatmap * 0.3 + img * 0.5
    cv2.imwrite("./vis_result/{}_true_{}_pred_{}_{}.jpeg".format(image_name.split('.')[0], val_csv.loc[index_to_viz]["class"], pred_labels[preds.item()], model_name), result)

### Resnet: Normal

!["image"](./vis_result/NORMAL2-IM-0276-0001_true_normal_pred_normal_resnet152.jpeg)

### Resnet: Pneumonia (virus)

!["image"](./vis_result/person1657_virus_2864_true_pneumonia_pred_pneumonia_resnet152.jpeg)

### Resnet: Pneumonia (bacteria)

!["image"](./vis_result/person134_bacteria_643_true_pneumonia_pred_pneumonia_resnet152.jpeg)

## DenseNet

In [0]:
# define the model
densenet161 = models.densenet161(pretrained=True)
densenet161 = models.densenet161(num_classes=2)

finalconv_name = 'features'
model_name = 'densenet161'

# training process
model = densenet161

LOAD_MODEL = True
MODEL_PATH = 'best_models/densenet161_best_model_198.pth'

if LOAD_MODEL:
    model.load_state_dict(torch.load(MODEL_PATH))

model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

### Densenet: Normal

!["image"](./vis_result/NORMAL2-IM-0276-0001_true_normal_pred_normal_densenet161.jpeg)

### Densenet: Pneumonia (virus)

!["image"](./vis_result/person1657_virus_2864_true_pneumonia_pred_pneumonia_densenet161.jpeg)

### Densenet: Pneumonia (bacteria)

!["image"](./vis_result/person134_bacteria_643_true_pneumonia_pred_pneumonia_densenet161.jpeg)

## GoogLeNet

In [0]:
# define the model
googlenet = models.googlenet(pretrained=True)
googlenet = models.googlenet(num_classes=2)
googlenet.aux_logits = False

finalconv_name = 'inception5b'
model_name = 'googlenet'

# training process
model = googlenet

root_dir = '/home/yz6121/dl/data/data/'
LOAD_MODEL = True
MODEL_PATH = root_dir+'best_gn21_model_.pth'

if LOAD_MODEL:
    model.load_state_dict(torch.load(MODEL_PATH))

model.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

### GoogLeNet: Normal

!['normal'](./vis_result/NORMAL2-IM-0276-0001_true_normal_pred_normal_googlenet.jpeg)

### GoogLeNet: Pnuemonia(virus)

!['virus'](./vis_result/person1657_virus_2864_true_pneumonia_pred_pneumonia_googlenet.jpeg)

### GoogLeNet: Pneumonia(bacteria)

!['bacteria'](./vis_result/person134_bacteria_643_true_pneumonia_pred_pneumonia_googlenet.jpeg)

# Test set(2 classes)
### GoogLeNet: Normal 

In [ ]:
test_csv = pd.read_csv("chest_xray_origin/all/test.csv")
# index = 0: penumonia(virus)
# index = 1: penumonia(bacteria)
# index = 3: normal

!['Normal'](./vis_result/test_pics/2classNORMAL2-IM-0359-0001_true_normal_pred_normal_googlenet.jpeg)

### GoogLeNet: Pneumonia(virus)

!['pneumonia'](./vis_result/test_pics/2classperson1651_virus_2855_true_pneumonia_pred_pneumonia_googlenet.jpeg)

### GoogLeNet: Pneumonia(bacteria)

!['penumonia(bac)'](./vis_result/test_pics/2classperson100_bacteria_478_true_pneumonia_pred_pneumonia_googlenet.jpeg)

# Test set(3 classes)


In [0]:
# define the model
googlenet = models.googlenet(pretrained=True)
googlenet = models.googlenet(num_classes=3)
googlenet.aux_logits = False

finalconv_name = 'inception5b'
model_name = 'googlenet'

# training process
model = googlenet

root_dir = '/home/yz6121/dl/data/data/'
LOAD_MODEL = True
MODEL_PATH = root_dir+'best_3class1.pth'

if LOAD_MODEL:
    model.load_state_dict(torch.load(MODEL_PATH))

model.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [ ]:
test_csv = pd.read_csv("chest_xray_origin/all/test_3.csv")

### GoogLeNet: Normal

!['normal'](./vis_result/test_pics/3classNORMAL2-IM-0359-0001_true_normal_pred_normal_googlenet.jpeg)

### GoogLeNet: Virus

!["virus"](./vis_result/test_pics/3classperson1651_virus_2855_true_virus_pred_virus_googlenet.jpeg)

### GoogLeNet: Bacteria

!['bacteria'](./vis_result/test_pics/3classperson100_bacteria_478_true_bacteria_pred_bacteria_googlenet.jpeg)